In [ ]:
import dicom
import os, sys, cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
url = '../input/sample_images/0a0c32c9e08cc2ea76a71649de56be6d'
slices_names = os.listdir(url)

scan_data = []

for slice_name in slices_names:
	slice = dicom.read_file(url+'/'+slice_name)
	scan_data.append(slice)

scan_data.sort(key=lambda z: z.SliceLocation)

scan = []
for i in range(len(scan_data)):
	scan.append((scan_data[i].pixel_array * scan_data[i].RescaleSlope) + scan_data[i].RescaleIntercept)

scan = np.array(scan)

In [ ]:
def segment_lung(scan):
	s = np.zeros(scan.shape)
	for i in range(scan.shape[0]):
		slice = np.array(scan[i]).reshape(scan.shape[1:])
		slice1 = slice.copy()
		slice1 = cv2.GaussianBlur(slice1, (5, 5), 0)

		slice1[slice1 > -300] = 255
		slice1[slice1 < -300] = 0
		slice1 = np.uint8(slice1)

		im2, contours, _ = cv2.findContours(slice1, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
		largest_contour = max(contours, key=cv2.contourArea)

		mask = np.zeros(slice.shape, np.uint8)
		cv2.fillPoly(mask, [largest_contour], 255)

		slice1 = ~slice1
		slice1[mask == 0] = 0

		kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
		slice1 = cv2.morphologyEx(slice1, cv2.MORPH_OPEN, kernel)  # <- to remove speckles...
		slice1 = cv2.morphologyEx(slice1, cv2.MORPH_DILATE, kernel)
		slice1 = cv2.morphologyEx(slice1, cv2.MORPH_DILATE, kernel)
		slice1 = cv2.morphologyEx(slice1, cv2.MORPH_CLOSE, kernel)
		slice1 = cv2.morphologyEx(slice1, cv2.MORPH_CLOSE, kernel)
		slice1 = cv2.morphologyEx(slice1, cv2.MORPH_ERODE, kernel)
		slice1 = cv2.morphologyEx(slice1, cv2.MORPH_ERODE, kernel)

		slice[slice1 == 0] = 0
		s[i, :, :] = slice

	return np.array(s)

In [ ]:
SCAN = segment_lung(scan)

In [ ]:
im = SCAN[70, :, :]
im = cv2.GaussianBlur(im, (3, 3), 0)
im[im > 0] = 0
im[im < -800] = 0
fig = plt.figure(figsize=(13, 13))
plt.imshow(im, cmap='gray')
plt.show()